# Stream Data Pipeline Exercise using Apache Kafka
This Colab is used to learn Kafka to create an example project.

### Goal: Create a Streaming Data Pipeline using Kafka

### Steps:

* **Step 1**: Create a Kafka Producer and Consumer to send and consume any sample data.
  * Please note that creating this pipeline in the environment of Google Colab is not ideal, but we do this as an illustrative purpose based on the architecture we covered during the lecture.
* **Step 2**: Fetch data from yfinance (i.e., Yahoo Finance) for one day and send the data from Producer and conume via Consumer.
* **Step 3**: After consuming the data in Kafka Consumer, find the rolling average for analysis.
* **Step 4**: Plot the data after finding the rolling average.



## Kafka Producer
KafkaProducer is a high-level, asynchronous message producer.

In [ ]:
# Python client for the Apache Kafka distributed stream processing system.
!pip3 install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
# Avro is an open source project that provides data serialization
# and data exchange services for Apache Hadoop. These services can be used together or independently.
# Avro facilitates the exchange of big data between programs written in any language.
# https://cwiki.apache.org/confluence/display/AVRO/Index
!pip install avro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for avro: filename=avro-1.11.3-py2.py3-none-any.whl size=123912 sha256=51e8e6e8b63b9a8d570aee1ccf505cd13a59307cfe1f6c0d574e0d183a7efdf1
  Stored in directory: /root/.cache/pip/wheels/1d/f6/41/0e0399396af07060e64d4e32c8bd259b48b98a4a114df31294
Successfully built avro


In [ ]:
import time
import json
import kafka
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [ ]:
import io
import avro.schema
from avro.io import DatumWriter

In [ ]:
# the code below mounts your google drive to /content/drive folder
# you might need to provide authorizations using your google account
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Download the stock schema file from [here](https://drive.google.com/uc?export=download&id=19QGksQA7wFa3SZHAz1h4b3zzXFfLRHvg).

In [ ]:
# With Schema
SCHEMA_PATH = "/content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Assignment3/stock_schema.avsc"
SCHEMA = avro.schema.parse(open(SCHEMA_PATH).read())

'''
{
	"namespace": "stock_schema.avro",
	"type": "record",
	"name": "Stock",
	"fields": [
		{"name": "name", "type": "string"},
		{"name": "time",  "type": "string"},
		{"name": "price", "type": "float"}
	]
}
'''

'\n{\n\t"namespace": "stock_schema.avro",\n\t"type": "record",\n\t"name": "Stock",\n\t"fields": [\n\t\t{"name": "name", "type": "string"},\n\t\t{"name": "time",  "type": "string"},\n\t\t{"name": "price", "type": "float"}\n\t]\n}\n'

**cURL** stands for Client URL, a lightweight command line tool for making network requests from the client side. It's suitable for many use cases, like making HTTP requests and testing APIs.

**xzf**, XTRACT ZE FILES!, is a utility for file extraction.

In [ ]:
# Download Apache Kafka and unzip into your Google Colab session.
!curl -sSOL https://archive.apache.org/dist/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz

In [ ]:
# start zookeeper server & kafka server set $KAFKA_HOME variable path in bashrc (i.e., a configuration file for the Bash shell).
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties #--override delete.topic.enable=true
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


The command-line interface (CLI) unquestionably acts as an impressive tool for both developers and system administrators due to its ability to identify and manage specific processes.

The ```ps``` command is a powerful tool that provides **information about the running processes**. Moreover, this command, along with other helper commands such as ```grep```, can **identify and earmark the Python processes in particular.**

First, ```ps -ef``` retrieves a list of all running processes in detail. The ```|``` **redirects the output** of the ```ps``` command **as the input to the next command.** The ```grep``` command searches through the input for lines that are related to kafka.

In [ ]:
# see if kafka properties are shown in the results.
!ps -ef | grep kafka

root        1406       1 16 11:43 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.1.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.1.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.1.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.1.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.1.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.1.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.1.0/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.13-3.1.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.1.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.13

In [ ]:
# create topics
# define producer
topic = 'tesla'
bootstrap_server = "localhost:9092"
producer = kafka.KafkaProducer(
    bootstrap_servers=[bootstrap_server]
)

### Get Stock Live Data from Yahoo Finance

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import datetime

In [ ]:
# Getting wierd graphs since there a gap between days
# df = yf.download(tickers='UBER', period='5d', interval='5m') // Modify period and interval based on you preference.

ticker = 'TSLA'
df = yf.download(tickers=ticker, period='1d', interval='5m')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
df.head(5)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-10-09 09:30:00-04:00,255.309998,256.619995,253.750000,254.320007,254.320007,5642471
2023-10-09 09:35:00-04:00,254.300003,254.449905,253.470001,254.034805,254.034805,2735857
2023-10-09 09:40:00-04:00,254.005005,255.550003,253.699997,255.160004,255.160004,2742137
2023-10-09 09:45:00-04:00,255.110001,256.079987,254.300003,255.990005,255.990005,2462230
2023-10-09 09:50:00-04:00,256.030914,256.299988,255.160004,255.869904,255.869904,2372790


In [ ]:
stock_dict = df["High"].to_dict().items()

In [ ]:
for keys,values in stock_dict:
    print(keys)
    print(values)

2023-10-09 09:30:00-04:00
256.6199951171875
2023-10-09 09:35:00-04:00
254.4499053955078
2023-10-09 09:40:00-04:00
255.5500030517578
2023-10-09 09:45:00-04:00
256.0799865722656
2023-10-09 09:50:00-04:00
256.29998779296875
2023-10-09 09:55:00-04:00
256.4700012207031
2023-10-09 10:00:00-04:00
256.32000732421875
2023-10-09 10:05:00-04:00
255.1199951171875
2023-10-09 10:10:00-04:00
255.08999633789062
2023-10-09 10:15:00-04:00
255.47999572753906
2023-10-09 10:20:00-04:00
255.72000122070312
2023-10-09 10:25:00-04:00
255.55850219726562
2023-10-09 10:30:00-04:00
255.24000549316406
2023-10-09 10:35:00-04:00
254.639892578125
2023-10-09 10:40:00-04:00
254.42999267578125
2023-10-09 10:45:00-04:00
254.5399932861328
2023-10-09 10:50:00-04:00
253.5800018310547
2023-10-09 10:55:00-04:00
253.80999755859375
2023-10-09 11:00:00-04:00
253.25999450683594
2023-10-09 11:05:00-04:00
252.59500122070312
2023-10-09 11:10:00-04:00
252.7198944091797
2023-10-09 11:15:00-04:00
252.9499969482422
2023-10-09 11:20:00-04

### KafkaProducer API

In order to send messages asynchronously to a topic, KafkaProducer class provides send method. So, the signature of ```send()``` is:

```python
producer.send(new ProducerRecord<byte[],byte[]>(topic,
partition, key1, value1) , callback);
```

In [ ]:
# With Schema
try:
    for d in stock_dict:

        bytes_writer = io.BytesIO() # Python BytesIO: Just like what we do with variables,
                                    # data can be kept as bytes in an in-memory buffer when we use the io module’s Byte IO operations.
        encoder = avro.io.BinaryEncoder(bytes_writer)

        writer = DatumWriter(SCHEMA)
        data = {
            "name": ticker,
            "time": d[0].strftime('%Y-%m-%d %H:%M:%S'),
            "price": d[1]
        }
        writer.write(data, encoder)

        raw_bytes = bytes_writer.getvalue()
        print(data, raw_bytes)

        # producer
        producer.send(topic, raw_bytes)
        # print(producer.send(topic, raw_bytes).get(timeout=10).topic)
        # print(producer.send(topic, raw_bytes).get(timeout=10).partition)
        # print(producer.send(topic, raw_bytes).get(timeout=10).offset)

        # time.sleep(1)
        time.sleep(0)
except Exception as e:
    print("We recevied a error:::")
    print(e)
finally:
    print("<---------------->")

{'name': 'TSLA', 'time': '2023-10-09 09:30:00', 'price': 256.6199951171875} b'\x08TSLA&2023-10-09 09:30:00\\O\x80C'
{'name': 'TSLA', 'time': '2023-10-09 09:35:00', 'price': 254.4499053955078} b'\x08TSLA&2023-10-09 09:35:00-s~C'
{'name': 'TSLA', 'time': '2023-10-09 09:40:00', 'price': 255.5500030517578} b'\x08TSLA&2023-10-09 09:40:00\xcd\x8c\x7fC'
{'name': 'TSLA', 'time': '2023-10-09 09:45:00', 'price': 256.0799865722656} b'\x08TSLA&2023-10-09 09:45:00=\n\x80C'
{'name': 'TSLA', 'time': '2023-10-09 09:50:00', 'price': 256.29998779296875} b'\x08TSLA&2023-10-09 09:50:00f&\x80C'
{'name': 'TSLA', 'time': '2023-10-09 09:55:00', 'price': 256.4700012207031} b'\x08TSLA&2023-10-09 09:55:00)<\x80C'
{'name': 'TSLA', 'time': '2023-10-09 10:00:00', 'price': 256.32000732421875} b'\x08TSLA&2023-10-09 10:00:00\xf6(\x80C'
{'name': 'TSLA', 'time': '2023-10-09 10:05:00', 'price': 255.1199951171875} b'\x08TSLA&2023-10-09 10:05:00\xb8\x1e\x7fC'
{'name': 'TSLA', 'time': '2023-10-09 10:10:00', 'price': 255.089

The data produced by a producer is asynchronous. Therefore, two additional functions, i.e., ```flush()``` and ```close()``` are required to ensure the producer is shut down after the message is sent to Kafka. The ```flush()``` will force all the data that was in ```send()``` to be produced and ```close()``` stops the producer.

In [ ]:
# producer.flush()
# producer.close()

In [ ]:
## In many cases, we define the functions like below to "write to kafka" for specific topics using producer.
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  global count
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message in items:

        bytes_writer = io.BytesIO()
        encoder = avro.io.BinaryEncoder(bytes_writer)

        writer = DatumWriter(SCHEMA)
        data = {
            "name": ticker,
            "time": message[0].strftime('%Y-%m-%d %H:%M:%S'),
            "price": message[1]
        }
        writer.write(data, encoder)

        raw_bytes = bytes_writer.getvalue()
        print(data, raw_bytes)
        # producer.send(topic_name)
        producer.send(topic_name, raw_bytes)
        # producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
        count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))


  # for message, key in items:
  #   producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
  #   count+=1
  # producer.flush()
  # print("Wrote {0} messages into topic: {1}".format(count, topic_name))

In [ ]:
write_to_kafka(topic, stock_dict)

{'name': 'TSLA', 'time': '2023-10-09 09:30:00', 'price': 256.6199951171875} b'\x08TSLA&2023-10-09 09:30:00\\O\x80C'
{'name': 'TSLA', 'time': '2023-10-09 09:35:00', 'price': 254.4499053955078} b'\x08TSLA&2023-10-09 09:35:00-s~C'
{'name': 'TSLA', 'time': '2023-10-09 09:40:00', 'price': 255.5500030517578} b'\x08TSLA&2023-10-09 09:40:00\xcd\x8c\x7fC'
{'name': 'TSLA', 'time': '2023-10-09 09:45:00', 'price': 256.0799865722656} b'\x08TSLA&2023-10-09 09:45:00=\n\x80C'
{'name': 'TSLA', 'time': '2023-10-09 09:50:00', 'price': 256.29998779296875} b'\x08TSLA&2023-10-09 09:50:00f&\x80C'
{'name': 'TSLA', 'time': '2023-10-09 09:55:00', 'price': 256.4700012207031} b'\x08TSLA&2023-10-09 09:55:00)<\x80C'
{'name': 'TSLA', 'time': '2023-10-09 10:00:00', 'price': 256.32000732421875} b'\x08TSLA&2023-10-09 10:00:00\xf6(\x80C'
{'name': 'TSLA', 'time': '2023-10-09 10:05:00', 'price': 255.1199951171875} b'\x08TSLA&2023-10-09 10:05:00\xb8\x1e\x7fC'
{'name': 'TSLA', 'time': '2023-10-09 10:10:00', 'price': 255.089

# Kafka Consumer

In [ ]:
# ! /content/kafka_2.13-3.1.0/bin/kafka-console-consumer.sh \
# --bootstrap-server localhost:9092 \
# --topic tesla \
# --from-beginning --max-messages 3

In [ ]:
# define consumer
bootstrap_server = "localhost:9092"
group_id = "my_stock"
consumer = kafka.KafkaConsumer(
    bootstrap_servers=[bootstrap_server],
    group_id = group_id
)

In [ ]:
# subsribe to a specified topic
topic = 'tesla'
consumer.subscribe(topics=[topic])
consumer.subscription()

{'tesla'}

In [ ]:
# Streamlit is an open-source app framework for Machine Learning and Data Science teams. Create beautiful web apps in minutes.
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
import time
import json
import datetime
import kafka
import streamlit as st

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import io
import avro.schema
import avro.io

In [ ]:
# define schema for a dataframe in which we store moving average.
schema = {
    "time": [],
    "price": [],
    "moving_avg": []
}
df = pd.DataFrame(schema)

In [ ]:
## Generating one plot as an exercise (interrupt if you don't see updates any more):

# try:
#   for message in consumer:
#       bytes_reader = io.BytesIO(message.value)
#       decoder = avro.io.BinaryDecoder(bytes_reader)
#       reader = avro.io.DatumReader(SCHEMA)
#       stock_data = reader.read(decoder)
#       print(stock_data)

#       # creating rolling avg
#       roll_avg = np.nan
#       if len(df) >= 5:
#           roll_avg = df["price"].iloc[-5:].mean()
#       else:
#           roll_avg = df["price"].iloc[-len(df):].mean()
#       print("roll_avg: ", roll_avg)

#       # Adding record to DataFrame
#       timestamp = datetime.datetime.strptime(stock_data["time"], "%Y-%m-%d %H:%M:%S")
#       price = stock_data["price"]
#       df.loc[len(df.index)] = [timestamp, price, roll_avg]

# except KeyboardInterrupt:
#     print("process interrupted")

In [ ]:
## After the interruption, run this part to plot price and moving average.
## Time series plot with Seaborn lineplot()

# fig = plt.figure(figsize=(9,6))
# # Time series plot with Seaborn lineplot()
# plt.plot(df["time"], df["price"], 'k.-', label='Original data')
# plt.plot(df["time"], df["moving_avg"], 'r.-', label='Running average')
# # axis labels
# plt.xlabel("Date", size=14)
# plt.ylabel("Price", size=14)
# # save image as PNG file
# plt.savefig("/content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/StockPlots/Stock_Price_Chart.png",
#                     format='png',
#                     dpi=150)
# st.pyplot(fig)

### How does Kafka Avro deserializer work?

The Kafka Avro Deserializer in the consumer gets the schema Id from the message and uses this to look up the schema from the Schema Registry. The message is deserialized, verified using the retrieved schema.

In [ ]:
# app.py

# Decommentize below line to write app.py file:
# %%writefile "/content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/StockPlots/app.py"

import streamlit as st # web development
import numpy as np # np mean, np random
import pandas as pd # read csv, df manipulation
import time # to simulate a real time data, time loop
import plotly.express as px # interactive charts

# dashboard title
st.set_page_config(
    page_title = 'Real-Time Data Science Dashboard',
    page_icon = '✅',
    layout = 'wide'
)

# st.title("Stock Moving Average Dashboard")

# creating a single-element container.
placeholder = st.empty()

try:
  count_fig = 0
  for message in consumer:

      bytes_reader = io.BytesIO(message.value)
      decoder = avro.io.BinaryDecoder(bytes_reader)
      reader = avro.io.DatumReader(SCHEMA)
      stock_data = reader.read(decoder)
      # print(stock_data)

      # creating rolling avg
      roll_avg = np.nan
      if len(df) >= 5:
          roll_avg = df["price"].iloc[-5:].mean()
      else:
          roll_avg = df["price"].iloc[-len(df):].mean()
      # print("roll_avg: ", roll_avg)

      # Adding record to DataFrame
      timestamp = datetime.datetime.strptime(stock_data["time"], "%Y-%m-%d %H:%M:%S")
      price = stock_data["price"]
      df.loc[len(df.index)] = [timestamp, price, roll_avg]

      with placeholder.container():

          ## Section 1
          # create two columns for Price and Rolling Avg
          kpi1, kpi2 = st.columns(2)

          # fill in those three columns with respective metrics or KPIs
          kpi1.metric(label="Price: ", value=round(price, 2), delta=round(price - df["price"].get(len(df)-2, 0), 4))
          kpi2.metric(label="Rolling_Avg", value= round(roll_avg, 2))

          ## Section 2
          st.markdown("Stock Graph with Moving average")

          fig = plt.figure(figsize=(9,6))
          # Time series plot with Seaborn lineplot()
          plt.plot(df["time"], df["price"], 'k.-', label='Original data') # draw a black line.
          plt.plot(df["time"], df["moving_avg"], 'r.-', label='Running average') # draw a red line.
          # axis labels
          plt.xlabel("Date", size=14)
          plt.ylabel("Price", size=14)
          # save image as PNG file
          plt.savefig("/content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/StockPlots/Stock_Price_Chart_{}.png".format(count_fig),
                              format='png',
                              dpi=150)
          st.write(fig)
          count_fig += 1
          # print("current count: ",count)
          st.pyplot(fig)
          plt.show()
          fig.clear()

          ### KeyboardInterruptInterrupt if count_fig is the same as the max # moving average available.
          if count_fig == count-4: #
              raise KeyboardInterrupt

          ## Section 3
          st.markdown("### Detailed Data View")
          st.dataframe(df.iloc[-5:])
          time.sleep(1)

except KeyboardInterrupt:
    print("process interrupted")

In [ ]:
# !streamlit run /content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/StockPlots/app.py &>/content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/StockPlots/logs.txt &


In [ ]:
# !npx localtunnel --port 8501 & curl ipv4.icanhazip.com

## How to Launch Streamlit App from Google Colab Notebook: [link](https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399/1)

In [ ]:
# Install localtunnel to serve the Streamlit app
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.748s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
# Run the Streamlit app in the background
# e.g., !streamlit run /content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/app.py &>/content/drive/MyDrive/Colab Notebooks/MIS584 FA2023/Module3/StockPlots/logs.txt &
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
# Expose the Streamlit app on port 8501
# Then just click in the url showed.
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com